In [ ]:
# Set up the workspace
using SumOfSquares, JuMP, PolyJuMP, DynamicPolynomials, MultivariatePolynomials
using Mosek#, CSDP#, SCS
using Plots
gr()

include("../src/NormalSoS.jl")
using NormalSoS

# Linear Examples

As a first test case we will examine the linear example of Zhou _et al._ (2012). For a linear system the dynamics is expressed in terms of a matrix $A$, and the properties of the decomposition are determined by some well known properties of this matrix.
\begin{equation}
    \dot{x} = f(x) = Ax.
\end{equation}
If $A$ is a normal matrix, then an orthogonal decomposition should be achieveable via a symmetric-antisymmetric decomposition. In this case the quasi-potential is given by:
\begin{equation}
    V(x) = -\frac{1}{2}((A+A^*)x,x),
\end{equation}
where the brackets denote an inner product.

If $A$ is not normal, then the quasipotential is given by the slightly more complicated expression:
\begin{equation}
    V(x) = -\frac{1}{2}\left( \left( \int_0^\infty \exp(At) \exp(A^*t)\,\text{d}t\right)^{-1}x, x \right).
\end{equation}
Such a quasipotential will not satisfy a normal decomposition. A useful comparison can therefore be made between this analytical expression for the quasipotential and that resulting from the SoS method for non-normal linear systems.

In [ ]:
# A 2D normal example - WORKS
n = 2;    @polyvar x[1:n]
A = [-5.0 0.2;
     0.2 -1.0];
F1(X::Vector) = A*X;
f1 = F1(x);
U1 = NormalSoS.normdecomp(f1,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V1 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [6]:
@show(U1);    @show(V1);

U1 = 2.499999995607779x1^2 - 0.19999999229723395x1x2 + 0.49999999447044546x2^2 + 2.69039587883021
V1 = 2.5x1^2 - 0.2x1x2 + 0.5x2^2


In [ ]:
# A 3D normal example - WORKS
n = 3;    @polyvar x[1:n]
A = [-5.0 0.0 0.2;
     0.0 -1.5 0.0;
     0.2 0.0 -1.0];
F2(X::Vector) = A*X;
f2 = F2(x);
U2 = NormalSoS.normdecomp(f2,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V2 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [8]:
@show(U2);    @show(V2);

U2 = 2.4999998459624364x1^2 - 0.1999996920830655x1x3 + 0.7499999729568393x2^2 + 0.4999997328638468x3^2 + 1.799493653406083
V2 = 2.4999999999999996x1^2 - 0.1999999999999999x1x3 + 0.75x2^2 + 0.5x3^2


In [ ]:
# A 2D non-normal example - WORKS
n = 2;    @polyvar x[1:n]
A = [-5.0 3.8;
     0.2 -1.0];
F3(X::Vector) = A*X;
f3 = F3(x);
U3 = NormalSoS.normdecomp(f3,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
P = diagm(λ);
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V3 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [10]:
@show(U3);    @show(V3);
NormalSoS.checknorm(f3,U3,x)

U3 = 1.7941171716143742x1^2 - 2.3529404775815133x1x2 + 1.2058820662574214x2^2 + 2.9510708010743074
V3 = 1.7941176470588243x1^2 - 2.3529411764705888x1x2 + 1.2058823529411766x2^2


2.090979015415921e-6

In [ ]:
# A 3D non-normal example - WORKS and agrees with quasipotential
n = 3;    @polyvar x[1:n]
A = [-5.0 3.0 0.2;
     -1.0 -1.5 3.0;
     0.2 -1.0 -1.0];
F4(X::Vector) = A*X;
f4 = F4(x);
U4 = NormalSoS.normdecomp(f4,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V4 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [12]:
@show(U4);    @show(V4);
# @show(U4-V4)
NormalSoS.checknorm(f4,U4,x)
# λ

U4 = 1.8342917730973012x1^2 - 1.749788550729352x1x2 + 0.18785624780097818x1x3 + 0.9937841317069048x2^2 - 0.7118287002496625x2x3 + 0.9219234571078182x3^2 + 2.010661893647271
V4 = (1.8342920720248248 + 9.656330165015786e-17im)x1^2 + (-1.7497888702168567 - 4.089691322452249e-17im)x1x2 + (0.1878561059350676 - 2.8686731123587087e-17im)x1x3 + (0.9937843646267177 + 3.467670518240395e-17im)x2^2 + (-0.7118288495863689 - 1.0674447438687807e-16im)x2x3 + (0.9219235633484566 + 3.828799556799836e-17im)x3^2


8.58380831906698e-7

In [ ]:
# A 5D non-normal example with oscillatory dynamics - Orthogonal but does not match quasipotential
n = 5;    @polyvar x[1:n]
A = [-5.0 3.0 0.2 0.0 1.5;
     0.0 -1.5 0.0 0.0 0.5;
     -0.2 0.0 -1.0 0.0 1.0;
     0.0 1.2 0.0 -0.5 0.0;
     1.5 0.0 -3.0 0.0 -1.0];
F5(X::Vector) = A*X;
f5 = F5(x);
U5 = NormalSoS.normdecomp(f5,x, MosekSolver(),0);

(λ,u) = eig(A);
(~,v) = eig(A');
M = inv(u)*v
for i=1:n, j=1:n
    M[i,j] = -M[i,j]/(λ[i]+λ[j]);
end
V5 = NormalSoS.filterterms(0.25*dot(inv(u*M*inv(v))*x,x));

In [14]:
@show(U5);    @show(V5);
@show(NormalSoS.checknorm(f5,U5,x));
# λ

U5 = 2.0494185880556817x1^2 - 1.8125099996917033x1x2 - 0.7676737776541611x1x3 - 0.16803931591424467x1x4 - 1.1971199599911695x1x5 + 1.2180778436122575x2^2 + 0.057751565533307334x2x3 - 0.14689491711058086x2x4 + 0.1518972469851035x2x5 + 0.6496139544802867x3^2 - 0.04676495649634916x3x4 + 0.3083944765843497x3x5 + 0.16907830114542752x4^2 + 0.05220978579905337x4x5 + 0.4138089010726228x5^2 + 1.5672295443771338
V5 = (1.065138964391837 + 1.0870199733839228im)x1^2 + (0.8271661493964746 + 1.5481999588139224im)x1x2 + (0.20222927726671547 - 1.3378650659653184im)x1x3 + (-0.8575307140500827 + 0.3377299622424908im)x1x4 + (-1.6447163443901764 + 1.9601914686796746im)x1x5 + (2.3077638496401462 - 0.12869887051909554im)x2^2 + (0.03232186145462679 + 0.8302648905649072im)x2x3 + (-1.9843778205714488 - 0.13848091397223497im)x2x4 + (1.009733755488397 - 1.0071393355560114im)x2x5 + (-0.7282760558751338 + 0.06395869312739921im)x3^2 + (0.715549354843763 + 0.11773814127133025im)x3x4 + (1.1430751845959244 + 1.43186724

3.136579992753051e-6

In [ ]:
# Display simulations of the 5 state system with damped oscillatory dynamics
using DifferentialEquations

u0 = [1.0;1.0;1.0;1.0;1.0];
tspan = (0.0,10.0);
f(u,p,t) = F5(u);
prob = ODEProblem(f,u0,tspan);
sol = DifferentialEquations.solve(prob);

In [ ]:
plot(sol)